In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from fake_useragent import UserAgent
from datetime import datetime
import time


In [2]:
ua = UserAgent()
headers = {"User-Agent":ua.random}

In [3]:
local_time = datetime.now().strftime('%Y-%m-%d')

smtv_sports_url = "https://smtv.io/program/listwithgame/default?parent=43&child=all&day_length=7&date="+local_time+"&now=Y&filter_live=Y"
smtv_entertainment_url = "https://smtv.io/program/listwithgame/default?parent=44&child=all&day_length=7&date="+local_time+"&now=Y&filter_live=Y"
smtv_game_url = "https://smtv.io/program/listwithgame/default?parent=81&child=all&day_length=7&date="+local_time+"&now=Y&filter_live=Y"
smzb_sports_url = "https://smzb.io/program/listwithgame/default?parent=1&child=all&day_length=7&date="+local_time+"&now=Y&filter_live=Y"
smzb_entertainment_url = "https://smzb.io/program/listwithgame/default?parent=2&child=all&day_length=7&date="+local_time+"&now=Y&filter_live=Y"
smzb_game_url = "https://smzb.io/program/listwithgame/default?parent=45&child=all&day_length=7&date="+local_time+"&now=Y&filter_live=Y"
kl99_sports_url = "https://kl99.tv/program/listwithgame/default?parent=1&child=all&tag=&day_length=7&date="+local_time+"&now=Y&filter_live=Y"
lj88_sports_url = "https://lj88.tv/program/listwithgame/default?parent=27&child=all&tag=&day_length=7&date="+local_time+"&now=Y&filter_live=Y&from=gg"
ggzb_game_url = "https://ggzb.io/program/listwithgame/default?parent=1&child=all&tag=&day_length=7&date="+local_time+"&now=Y&filter_live=Y&from=gg"



In [4]:
# 網址

# smtv 體育
# https://smtv.io/program/43
# smtv 娛樂
# https://smtv.io/program/44
# smtv 電競
# https://smtv.io/program/81

# ------------------------

# smzb 體育
# https://smzb.io/program/1
# smzb 娛樂
# https://smzb.io/program/2
# smzb 電競    
# https://smzb.io/program/45

# ------------------------

# ggzb 電競
# https://ggzb.io/

# ------------------------

# lj88 體育
# https://lj88.tv/

# -----------------------------------------------------------------------


# 比較列表

# smtv體育 diff smzb體育
# smtv體育 diff kl99體育
# smtv體育 diff lj88體育

# ------------------------

# smtv電競 diff smzb電競
# smtv電競 diff ggzb電競

# ------------------------

# smtv娛樂 diff smzb娛樂


In [5]:
def Get_Schedule_to_DF(url):
    res = requests.get(url,headers=headers)
    schedule = json.loads(res.text)
    result = []
    for i in range(len(schedule["result"])):

        for j in range(len(schedule["result"][i]["data"])):

            live_types_name = schedule["result"][i]["data"][j]["live_types_name"]
            live_types_child_name = schedule["result"][i]["data"][j]["live_types_child_name"]
            lname = schedule["result"][i]["data"][j]["lname"]
            time = schedule["result"][i]["data"][j]["time"]
            hname = schedule["result"][i]["data"][j]["hname"]
            aname = schedule["result"][i]["data"][j]["aname"]
            account = []

            for k in schedule["result"][i]["data"][j]["account"]:
                nickname = k["nickname"]
                live_status = k["live_status"]

                if live_status == 1:
                    status = "開播"
                else:
                    status = "未開播"
                account.append({"nickname":nickname,"live_status":status})

            dict_={
                "live_types_name":live_types_name,
                "live_types_child_name":live_types_child_name,
                "lname":lname,
                "time":time,
                "hname":hname,
                "aname":aname,
                "account":account
            }
            result.append(dict_)

    df = pd.DataFrame(result)

    return df


In [6]:
# 含信號源
def Get_Schedule_diff_signal(url):
    res = requests.get(url,headers=headers)
    schedule = json.loads(res.text)
    result = []
    for i in range(len(schedule["result"])):

        for j in range(len(schedule["result"][i]["data"])):

            live_types_name = schedule["result"][i]["data"][j]["live_types_name"]
            live_types_child_name = schedule["result"][i]["data"][j]["live_types_child_name"]
            lname = schedule["result"][i]["data"][j]["lname"]
            time = schedule["result"][i]["data"][j]["time"]
            hname = schedule["result"][i]["data"][j]["hname"]
            aname = schedule["result"][i]["data"][j]["aname"]
            account = []

            for k in schedule["result"][i]["data"][j]["account"]:
                nickname = k["nickname"]
                live_status = k["live_status"]

                if live_status == 1:
                    status = "開播"
                else:
                    status = "未開播"
                account.append({"nickname":nickname,"live_status":status})

            result_str = time + "  " + lname + "  " + hname + " vs " + aname + "  " + str(account)


            result.append(result_str)

    return result

In [7]:
# 不含信號源
def Get_Schedule_diff(url):
    res = requests.get(url,headers=headers)
    schedule = json.loads(res.text)
    result = []
    for i in range(len(schedule["result"])):

        for j in range(len(schedule["result"][i]["data"])):

            live_types_name = schedule["result"][i]["data"][j]["live_types_name"]
            live_types_child_name = schedule["result"][i]["data"][j]["live_types_child_name"]
            lname = schedule["result"][i]["data"][j]["lname"]
            time = schedule["result"][i]["data"][j]["time"]
            hname = schedule["result"][i]["data"][j]["hname"]
            aname = schedule["result"][i]["data"][j]["aname"]

            result_str = time + "  " + lname + "  " + hname + " vs " + aname

            result.append(result_str)

    return result


In [8]:
def Schedule_diff(diff_A,diff_B):
    diff_1 = set(diff_A)
    diff_2 = set(diff_B)
    diff_result = sorted(list(diff_1.difference(diff_2)))
    return diff_result

In [9]:
# 比較賽事差異

print("----------以下為體育賽事----------")

smtv_sports = Get_Schedule_diff(smtv_sports_url)
smzb_sports = Get_Schedule_diff(smzb_sports_url)
diff_result = Schedule_diff(smtv_sports,smzb_sports)
print("分類:體育，smtv diff smzb")
print("數量:",len(diff_result))
for i in diff_result:
    print(i)
    
# print("==============================================")

# smtv_sports = Get_Schedule_diff(smtv_sports_url)
# kl99_sports = Get_Schedule_diff(kl99_sports_url)
# diff_result = Schedule_diff(smtv_sports,kl99_sports)
# print("分類:體育，smtv diff kl99")
# print("數量:",len(diff_result))
# for i in diff_result:
#     print(i)
    
print("==============================================")

smtv_sports = Get_Schedule_diff(smtv_sports_url)
lj88_sports = Get_Schedule_diff(lj88_sports_url)
diff_result = Schedule_diff(smtv_sports,lj88_sports)
print("分類:體育，smtv diff lj88")
print("數量:",len(diff_result))
for i in diff_result:
    print(i)
    
print("----------以下為娛樂直播----------")

smtv_entertainment = Get_Schedule_diff(smtv_entertainment_url)
smzb_entertainment = Get_Schedule_diff(smzb_entertainment_url)
diff_result = Schedule_diff(smtv_entertainment,smzb_entertainment)
print("分類:娛樂，smtv diff smzb")
print("數量:",len(diff_result))
for i in diff_result:
    print(i)
    
print("----------以下為電競賽事----------")

smtv_game = Get_Schedule_diff(smtv_game_url)
smzb_game = Get_Schedule_diff(smzb_game_url)
diff_result = Schedule_diff(smtv_game,smzb_game)
print("分類:電競，smtv diff smzb")
print("數量:",len(diff_result))
for i in diff_result:
    print(i)
    
print("==============================================")

smtv_game = Get_Schedule_diff(smtv_game_url)
ggzb_game = Get_Schedule_diff(ggzb_game_url)
diff_result = Schedule_diff(smtv_game,ggzb_game)
print("分類:電競，smtv diff ggzb")
print("數量:",len(diff_result))
for i in diff_result:
    print(i)
    




----------以下為體育賽事----------
分類:體育，smtv diff smzb
數量: 1
2020-06-30 16:00:00  国际友谊  奥比利克诺维女足 vs 伊康诺米斯女足
分類:體育，smtv diff lj88
數量: 33
2020-06-30 14:00:00  虚拟赛事  菲律宾 vs 土库曼
2020-06-30 16:00:00  国际友谊  奥比利克诺维女足 vs 伊康诺米斯女足
2020-06-30 22:45:00  伊朗甲  沙禾达利亚拉克 vs 塔比之
2020-06-30 22:45:00  伊朗甲  阿尔曼戈哈尔 vs 乌尔米耶
2020-06-30 23:00:00  国际友谊  塔波斯科 vs 皮耳森B队
2020-06-30 23:20:00  伊朗甲  厄尔布尔士 vs 法加尔瑟帕斯
2020-06-30 23:30:00  伊朗甲  达玛希吉兰 vs 塔拉依萨纳萨维
2020-06-30 23:30:00  意丙1B  艾摩勒斯 vs 阿尔齐尼亚诺
2020-07-01 00:00:00  意丙1B  AZ皮切尔诺 vs 雷恩德
2020-07-01 00:30:00  意丙1B  拉维纳 vs 阿尔玛祖云达斯法诺
2020-07-01 00:30:00  美足联(女)  华盛顿精神女足 vs 犹他王室女足
2020-07-01 01:00:00  土甲  艾丹亚斯普 vs 阿特诺度
2020-07-01 01:00:00  意丙1B  ASD西库拉 vs 比谢列1913
2020-07-01 01:00:00  瑞典甲  希尔维亚 vs 卡尔斯塔德
2020-07-01 01:00:00  英冠  卡迪夫城 vs 查尔顿
2020-07-01 01:00:00  英冠  巴恩斯利 vs 布莱克本
2020-07-01 01:00:00  英冠  雷丁 vs 布伦特福德
2020-07-01 01:30:00  以乙  普尔考凯 vs 雷霍沃特夏普尔
2020-07-01 01:30:00  英冠  女王公园巡游者 vs 富勒姆
2020-07-01 02:00:00  意丙1B  吉安纳 vs 奥尔比亚
2020-07-01 02:30:00  意丙1B  卡坦萨罗 vs 特华莫
2020-07-